In [1]:
import json
import os
import time
import ipyparallel as ipp

import matplotlib.pyplot as plt
import torch
import dfca as DFCA
import ifca as IFCA

import numpy as np

from util import *

Reads Config file and prepares the arguments you can choose in the config_dfca.json/config_ifca.json

In [2]:
rc = ipp.Cluster(n=2).start_and_connect_sync()
rc.activate()

rc.ids 

dview = rc[:]

Starting 2 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/2 [00:00<?, ?engine/s]

In [3]:
seeds = [1111,1222,1333,1444,1555]
learning_rates = [0.1]

dfca_gi = []
ifca = []

In [4]:
%%px --targets all

import sys
import os
sys.path.append(os.getcwd())

seeds = [1111,1222,1333,1444,1555]
learning_rates = [0.1]

dfca_gi = []
ifca = []

Running the dfca-gi experiment

In [5]:
%%px --target 0 --noblock
import dfca as DFCA
import json
import time
import torch
import numpy as np


with open("config_dfca_gi.json", "r") as read_file:
    config = json.load(read_file)

for lr in learning_rates:
    config['lr'] = lr
    res_final = []

    for seed in seeds:
        print(f"---------------------------------DFCA-GI, lr: {lr}, seed: {seed}---------------------------------")
        start_time = time.time()
        config['data_seed'] = seed
        config['train_seed'] = config['data_seed']

        print("config:",config)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {device}")

        exp = DFCA.TrainMNISTCluster(config, device)
        exp.setup()
        res = exp.run()
        res_final.append([[r['test']['loss'] for r in res], [r['test']['acc'] for r in res], [r['test']['cl_acc'] for r in res], [r['train']['loss'] for r in res]])
        del exp
        duration = (time.time() - start_time)
        print("---train cluster Ended in %0.2f hour (%.3f sec) " % (duration/float(3600), duration))

    res_mean = np.mean(res_final, axis=0).tolist()
    dfca_gi.append(res_mean)



<AsyncResult(%px): pending>

Running the ifca experiment

In [6]:
%%px --target 1 --noblock
import ifca as IFCA
import json
import time
import torch
import numpy as np

with open("config_ifca.json", "r") as read_file:
    config = json.load(read_file)

for lr in learning_rates:
    config['lr'] = lr
    res_final = []

    for seed in seeds:
        print(f"---------------------------------IFCA, lr: {lr}, seed: {seed}---------------------------------")
        start_time = time.time()
        config['data_seed'] = seed
        config['train_seed'] = config['data_seed']

        print("config:",config)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {device}")

        exp = IFCA.TrainMNISTCluster(config, device)
        exp.setup()
        res = exp.run()
        res_final.append([[r['test']['loss'] for r in res], [r['test']['acc'] for r in res], [r['test']['cl_acc'] for r in res], [r['train']['loss'] for r in res]])
        del exp
        duration = (time.time() - start_time)
        print("---train cluster Ended in %0.2f hour (%.3f sec) " % (duration/float(3600), duration))

    res_mean = np.mean(res_final, axis=0).tolist()
    ifca.append(res_mean)



<AsyncResult(%px): pending>

## Results

DFCA-GI Experiment Outputs

In [7]:
rc.wait_interactive()
_.display_outputs()

dfca_gi = rc[0].pull('dfca_gi', block=True)
ifca = rc[1].pull('ifca', block=True)

unknown:   0%|          | 0/2 [00:00<?, ?tasks/s]

Output for 1:
2025-06-01 21:35:09.148 [IPEngine] Loading connection info from $IPP_CONNECTION_INFO
2025-06-01 21:35:09.148 [IPEngine] WARNING | Not using CurveZMQ security
2025-06-01 21:35:09.150 [IPEngine] Registering with controller at tcp://127.0.0.1:44479
2025-06-01 21:35:09.153 [IPEngine.1] Shell_addrs: ['tcp://127.0.0.1:60071', 'tcp://127.0.0.1:48105', 'tcp://127.0.0.1:52121']
2025-06-01 21:35:09.153 [IPEngine.1] Connecting shell to tcp://127.0.0.1:60071
2025-06-01 21:35:09.153 [IPEngine.1] Connecting shell to tcp://127.0.0.1:48105
2025-06-01 21:35:09.153 [IPEngine.1] Connecting shell to tcp://127.0.0.1:52121
2025-06-01 21:35:09.153 [IPEngine.1] Starting nanny
2025-06-01 21:35:09.590 [KernelNanny.1] Starting kernel nanny for engine 1, pid=85604, nanny pid=85623
2025-06-01 21:35:09.590 [KernelNanny.1] Nanny watching parent pid 85604.
2025-06-01 21:35:09.662 [IPEngine.1] Loading IPython extension: storemagic
2025-06-01 21:35:09.664 [IPEngine.1] WARNING | debugpy_stream undefined, d

---------------------------------IFCA, lr: 0.1, seed: 1111---------------------------------
config: {'m': 4800, 'm_test': 800, 'p': 4, 'n': 50, 'uneven': True, 'h1': 200, 'num_epochs': 300, 'batch_size': 100, 'tau': 10, 'lr': 0.1, 'data_seed': 1111, 'train_seed': 1111, 'config_override': '', 'project_dir': 'output_ifca'}
Using device: cuda
Failed to download (trying next):
HTTP Error 404: Not Found

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Epoch -1 tr: l 2.286 a 0.145 clct[349, 936, 497, 3018] cl_acc 0.351  8.839sec
Epoch -1 tst: l 2.285 a 0.148 clct

100%|██████████| 9.91M/9.91M [00:01<00:00, 9.41MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 263kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 2.33MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.15MB/s]


Output for 0:
2025-06-01 21:35:09.053 [IPEngine] Loading connection info from $IPP_CONNECTION_INFO
2025-06-01 21:35:09.053 [IPEngine] WARNING | Not using CurveZMQ security
2025-06-01 21:35:09.055 [IPEngine] Registering with controller at tcp://127.0.0.1:44479
2025-06-01 21:35:09.059 [IPEngine.0] Shell_addrs: ['tcp://127.0.0.1:60071', 'tcp://127.0.0.1:48105', 'tcp://127.0.0.1:55847']
2025-06-01 21:35:09.059 [IPEngine.0] Connecting shell to tcp://127.0.0.1:60071
2025-06-01 21:35:09.059 [IPEngine.0] Connecting shell to tcp://127.0.0.1:48105
2025-06-01 21:35:09.059 [IPEngine.0] Connecting shell to tcp://127.0.0.1:55847
2025-06-01 21:35:09.059 [IPEngine.0] Starting nanny
2025-06-01 21:35:09.487 [KernelNanny.0] Starting kernel nanny for engine 0, pid=85601, nanny pid=85619
2025-06-01 21:35:09.487 [KernelNanny.0] Nanny watching parent pid 85601.
2025-06-01 21:35:09.559 [IPEngine.0] Loading IPython extension: storemagic
2025-06-01 21:35:09.561 [IPEngine.0] WARNING | debugpy_stream undefined, d

engine set stopped 1748806508: {'engines': {'1': {'exit_code': <Negsignal.SIGKILL: -9>, 'pid': 85604, 'identifier': '1'}, '0': {'exit_code': <Negsignal.SIGKILL: -9>, 'pid': 85601, 'identifier': '0'}}, 'exit_code': <Negsignal.SIGKILL: -9>}


NoEnginesRegistered: This operation requires engines. Try client.wait_for_engines(n) to wait for engines to register.

IFCA Experiment Outputs

In [8]:
__.display_outputs()

---------------------------------DFCA-GI, lr: 0.1, seed: 1111---------------------------------
config: {'m': 4800, 'm_test': 800, 'p': 4, 'n': 50, 'uneven': True, 'local_model_init': False, 'h1': 200, 'num_epochs': 300, 'batch_size': 100, 'tau': 10, 'lr': 0.1, 'data_seed': 1111, 'train_seed': 1111, 'config_override': '', 'project_dir': 'output_dfca_gi'}
Using device: cuda
Failed to download (trying next):
HTTP Error 404: Not Found

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Epoch -1 tr: l 2.286 a 0.145 clct[349, 936, 497, 3018] cl_acc 0.351  9.785sec
E

100%|██████████| 9.91M/9.91M [00:01<00:00, 9.03MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 241kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 2.36MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 3.70MB/s]


IFCA Experiment Outputs

## Plots

In [9]:

r_ifca = {}
r_dfca_gi = {}

r_ifca['test_loss'] = []
r_ifca['test_acc'] = []
r_ifca['cl_acc'] = []
r_ifca['train_loss'] = []
for i in range(len(learning_rates)):
    r_ifca['test_loss'].append(ifca[i][0])
    r_ifca['test_acc'].append(ifca[i][1])
    r_ifca['cl_acc'].append(ifca[i][2])
    r_ifca['train_loss'].append(ifca[i][3])

r_dfca_gi['test_loss'] = []
r_dfca_gi['test_acc'] = []
r_dfca_gi['cl_acc'] = []
r_dfca_gi['train_loss'] = []
for i in range(len(learning_rates)):
    r_dfca_gi['test_loss'].append(dfca_gi[i][0])
    r_dfca_gi['test_acc'].append(dfca_gi[i][1])
    r_dfca_gi['cl_acc'].append(dfca_gi[i][2])
    r_dfca_gi['train_loss'].append(dfca_gi[i][3])



os.makedirs('graphs', exist_ok=True)

for l in range(len(learning_rates)):

    plt.figure(figsize=(10,5))
    plt.plot(r_ifca['train_loss'][l], label='ifca', color='blue')
    plt.plot(r_dfca_gi['train_loss'][l], label='dfca-gi', color='green')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.title(f'Train Loss per Epoch (lr={learning_rates[l]})')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join('graphs', '1_train_loss.png'))

    plt.figure(figsize=(10,5))
    plt.plot(r_ifca['test_loss'][l], label='ifca', color='blue')
    plt.plot(r_dfca_gi['test_loss'][l], label='dfca-gi', color='green')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.title(f'Test Loss per Epoch (lr={learning_rates[l]})')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join('graphs', '1_test_loss.png'))

    plt.figure(figsize=(10,5))
    plt.plot(r_ifca['test_acc'][l], label='ifca', color='blue')
    plt.plot(r_dfca_gi['test_acc'][l], label='dfca-gi', color='green')
    plt.xlabel('epoch')
    plt.ylabel('test accuracy')
    plt.title(f'Test Accuracy per Epoch (lr={learning_rates[l]})')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join('graphs', '1_test_acc.png'))

    plt.figure(figsize=(10,5))
    plt.plot(r_ifca['cl_acc'][l][:31], label='ifca', color='blue')
    plt.plot(r_dfca_gi['cl_acc'][l][:31], label='dfca-gi', color='green')
    plt.xlabel('epoch')
    plt.ylabel('cluster acc')
    plt.title(f'Cluster Accuracy per Epoch (lr={learning_rates[l]})')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join('graphs', '1_cluster_acc.png'))

IndexError: list index out of range